In [ ]:
import numpy as np
import plotly.graph_objects as go
import csv
import scipy as sp
import sys,os

In [ ]:
def read_data(file_path):

    with open(file_path) as f:
        reader = csv.reader(f)
        data = []
        for column in reader:
            column[0] = column[0].replace('(', '').replace("'" , '')
            column[2] = column[2].replace(')', '').replace("'" , '')
            column[1] = column[1].replace("'" , '')
            column[0] = float(column[0])
            column[1] = float(column[1])
            column[2] = float(column[2])
            data.append(column)
            # print(data)
        data = np.array(data)
    return data
        # print(data)

In [ ]:
#Read energy in GeV of data files, save to list named energies
energies = []
#Read all files in data folder
for root, dir, files in os.walk('../data'):
    for file in files:
        if file.endswith('.csv'):
            energies.append(file.replace('.csv', '').replace('_', '.').replace('G', ''))
energies = np.array(energies, dtype=float)

In [ ]:
results = []
for root, dir, files in os.walk('../data'):
    for file in files:
        if file.endswith('.csv'):
            # print(file)
            data = read_data(os.path.join(root, file))
            results.append(data)

In [ ]:
for i, array in enumerate(results):
    print(f'data set{i} shape: {array.shape}')

In [25]:
def plot_pos(arrays, title, directory):
    # if not os.path.exists(directory):
    #     os.makedirs(directory)
    for i, array in enumerate(arrays):
        # np.savetxt(f'{directory}/array_{i}.csv', array, delimiter=',')
        # print(f"plotting array {array[:,0]}")
        energy_str = str(energies[i]).replace('.', '_')
        fig = go.Figure(data=go.Scatter(x=array[:,0], y=array[:,1], mode='markers'))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='x', yaxis_title='y')
        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1
            )
        )
        # fig.show()
        # fig.write_image('../media/plots/'+title+'.png')
        fig.write_image("../media/plots/"+f"{energies[i]}"+ 'G'+".png")


        # print("../media/plots/"+f"{energy_str}"+ 'G'+".png")

In [26]:
plot_dir = '../media/plots'
plot_pos(results, 'GeV', plot_dir)

KeyboardInterrupt: 

# Calculate emision angle

In [ ]:
def momentum(E, m):
    return np.sqrt(E**2 - m**2)

In [ ]:
def fit_circle(data):
    x, y = data[:,0], data[:,1]
    x_m, y_m = np.mean(x), np.mean(y)

    def calc_R(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((x-xc)**2 + (y-yc)**2)

    def f(c):
        """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = calc_R(*c)
        return Ri - Ri.mean()

    center_estimate = x_m, y_m
    center, _ = sp.optimize.leastsq(f, center_estimate)

    xc, yc = center
    Ri       = calc_R(*center)
    R       = Ri.mean()
    return xc, yc, R

In [ ]:
# distancia 0.24 m
d = 240 # mm
cherenkov_angle = []
for i, array in enumerate(results):
    theta = np.arctan(radii[i]/d)
    cherenkov_angle.append(theta)
    print(f'angle for {energies[i]} GeV: {theta}')
cherenkov_ang = np.array(cherenkov_angle)

print(cherenkov_ang)

In [ ]:
radii = []
xc = fit_circle(array)[0]
yc = fit_circle(array)[1]
momenta = []
# print(type(energies))
for e in range(len(energies)):
    p = momentum(energies[e], 0.139)
    print(energies[e])
    momenta.append(p)
momenta_array = np.array(momenta)
for array in results:
    radii.append(fit_circle(array)[2])
radii = np.array(radii)
print("momenta array", momenta_array.shape, "energies", energies.shape, "radios ",radii.shape)
# distancia 0.24 m
d = 240 # mm
cherenkov_angle = []
for i, array in enumerate(results):
    theta = np.arctan(radii[i]/d)
    cherenkov_angle.append(theta)
cherenkov_ang = np.array(cherenkov_angle)


processed_data = np.column_stack((energies, radii, momenta_array,cherenkov_ang))
print(processed_data.shape)

In [ ]:
def plot_circle(xc, yc, radii, arrays, energies):
    for i, array in enumerate(arrays):
        energy_str = str(energies[i]).replace('.', '_')
        theta_fit = np.linspace(0, 2*np.pi, 180)
        x_fit = xc + (radii[i])*np.cos(theta_fit)
        y_fit = yc + (radii[i])*np.sin(theta_fit)
        fig = go.Figure(data=go.Scatter(x=x_fit, y=y_fit, mode='lines', marker=dict(color='blue', opacity=0.8)))
        fig.add_trace(go.Scatter(x=array[:,0], y=array[:,1], mode='markers', marker=dict(color='violet', size=3, opacity=0.8)))
        fig.update_layout(title=f"{energy_str}"+ 'G', xaxis_title='y', yaxis_title='z')

        fig.update_layout(
            width=500,
            height=500,
            margin=dict(
                l=50,
                r=50,
                b=100,
                t=100,
                pad=4
            ),
            xaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1,
                scaleanchor="x",
                range=[-200, 200]
            ),
            yaxis=dict(
                showgrid=True,
                zeroline=True,
                showline=True,
                showticklabels=True,
                scaleratio=1,
                scaleanchor="y",
                range=[-200, 200]
            )
        )

        fig.write_image("../media/plots/"+"ang_p"+f"{energies[i]}"+ 'G'+".png")



In [ ]:
plot_circle(xc, yc, radii, results, energies)

In [ ]:
def plot_momenta_cherenkov_angle(data):
        fig = go.Figure(data=go.Scatter(x=data[:,2], y=data[:,3], mode='markers', marker=dict(color='darkgreen', opacity=0.8, size=2)))
        fig.update_layout(title='Cherenkov angle vs momenta', xaxis_title='momenta', yaxis_title='cherenkov angle', width=500, height=500)
        fig.show()

In [ ]:
plot_momenta_cherenkov_angle(processed_data)